In [ ]:
import os
import yaml
import time
import ccxt
import datetime
import calendar
import pandas as pd
import numpy as np
from stockstats import StockDataFrame as Sdf


from finrl.meta.data_processors.processor_ccxt import CCXTEngineer


TRAIN_START_DATE, TRAIN_END_DATE = "2022-06-01 00:00:00", "2025-01-01 00:00:00"
EVAL_START_DATE, EVAL_END_DATE = "2025-01-01 00:00:00", "2025-07-01 00:00:00"

processor = CCXTEngineer()
df = processor.data_fetch(TRAIN_START_DATE, EVAL_END_DATE, period="1h")
df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df[~df.index.duplicated(keep="last")]
print(df)


451 {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.bin


NetworkError: binance GET https://api.binance.com/api/v3/exchangeInfo

In [ ]:
INDICATORS = [
    'macd',          # MACD (默认 12, 26)
    'boll_ub',       # 布林带上轨 (默认 20)
    'boll_lb',       # 布林带下轨 (默认 20)
    'rsi_30',        # 30周期 RSI
    'dx_30',         # 30周期 DX (动向指标)
    'close_30_sma',  # 30周期收盘价简单移动平均
    'close_60_sma'   # 60周期收盘价简单移动平均
]
dataset = processor.add_technical_indicators(
    df,
    ["BTC/USDT"],
    INDICATORS,
)
dataset = dataset.iloc[100:]
print(dataset.isna().sum())
print(dataset)


Succesfully add technical indicators
BTC/USDT  open            0
          high            0
          low             0
          close           0
          volume          0
          macd            0
          boll_ub         0
          boll_lb         0
          rsi_30          0
          dx_30           0
          close_30_sma    0
          close_60_sma    0
dtype: int64
                      BTC/USDT                                              \
                          open       high        low      close     volume   
2022-06-05 12:00:00   29766.95   29782.19   29686.00   29722.90  646.57341   
2022-06-05 13:00:00   29722.90   29819.85   29722.89   29793.95  439.88382   
2022-06-05 14:00:00   29793.94   29796.83   29729.70   29729.71  376.08204   
2022-06-05 15:00:00   29729.71   29810.20   29645.67   29645.67  666.95377   
2022-06-05 16:00:00   29645.68   29738.73   29643.93   29691.81  615.97894   
...                        ...        ...        ...        ...     

In [ ]:
def slice_data(df, start_date, end_date):
    datetime = df.index
    if not isinstance(datetime, pd.DatetimeIndex):
        datetime = pd.to_datetime(datetime)
    datetime = datetime.sort_values()
        
    start_ts = pd.to_datetime(start_date)
    end_ts = pd.to_datetime(end_date)

    start_index = datetime.get_indexer([start_ts], method="backfill")[0]
    end_index = datetime.get_indexer([end_ts], method="pad")[0]

    if end_ts == datetime[end_index]:
        end_index -= 1

    return df.iloc[start_index: end_index]

train_df = slice_data(dataset, TRAIN_START_DATE, TRAIN_END_DATE)
eval_df = slice_data(dataset, EVAL_START_DATE, EVAL_END_DATE)
print(train_df.shape)
print(train_df)
print(eval_df)

(22570, 12)
                     BTC/USDT                                            \
                         open      high       low     close      volume   
2022-06-05 12:00:00  29766.95  29782.19  29686.00  29722.90   646.57341   
2022-06-05 13:00:00  29722.90  29819.85  29722.89  29793.95   439.88382   
2022-06-05 14:00:00  29793.94  29796.83  29729.70  29729.71   376.08204   
2022-06-05 15:00:00  29729.71  29810.20  29645.67  29645.67   666.95377   
2022-06-05 16:00:00  29645.68  29738.73  29643.93  29691.81   615.97894   
...                       ...       ...       ...       ...         ...   
2024-12-31 18:00:00  94023.88  94271.29  93664.06  94113.69   715.16946   
2024-12-31 19:00:00  94113.69  94285.12  93984.00  94232.68   540.22294   
2024-12-31 20:00:00  94232.68  94958.85  94232.67  94539.85  1252.18005   
2024-12-31 21:00:00  94539.86  96054.79  94524.43  95587.12  1803.50187   
2024-12-31 22:00:00  95587.12  96250.00  95372.35  95679.17  1504.33717   

            